In [1]:
from yolov5_predictor import Yolov5_Predictor
from unet_predictor import UNet_Predictor
import cv2
import numpy as np
import matplotlib.pyplot as plt


def iou(box0: np.ndarray, box1: np.ndarray):
    """
    Args:
        box0, box1: np.ndarray of shape (4, )
    """
    xy_max = np.minimum(box0[2:], box1[2:])
    xy_min = np.maximum(box0[:2], box1[:2])

    inter = np.clip(xy_max-xy_min, a_min=0, a_max=np.inf)
    inter = inter[0]*inter[1]

    area_0 = (box0[2]-box0[0])*(box0[3]-box0[1])
    area_1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    union = area_0 + area_1- inter
    return inter / union

class_names = ['powder_uncover', 'powder_uneven', 'scratch']

def load_img(img_path: str, binary=False):
    if not binary:
        return cv2.imread(img_path)
    else:
        im = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        # Binarize
        _ , im = cv2.threshold(im, 0, 255, cv2.THRESH_BINARY)
        return im

def dice_coeff(input: np.ndarray, target: np.ndarray, epsilon: float = 1e-6):
    # Average of Dice coefficient for all batches, or for a single mask
    assert input.shape == target.shape
    assert len(input.shape) == 2

    _, input = cv2.threshold(input, 0, 1, cv2.THRESH_BINARY)
    input = input.astype(np.int32)
    _, target = cv2.threshold(target, 0, 1, cv2.THRESH_BINARY)
    target = target.astype(np.int32)

    inter = 2 * (input * target).sum()
    sets_sum = input.sum() + target.sum()
    sets_sum = np.where(sets_sum == 0, inter, sets_sum)

    dice = (inter + epsilon) / (sets_sum + epsilon)
    return np.mean(dice)

class Powder_Spreading_Predictor(object):
    def __init__(self, yolo_weight_path:str, 
                        unet_weight_path:str,
                        bilinear=False,
                        crop_size=(320,320),
                        device:str = 'cpu'):
        self.yolov5_predictor = Yolov5_Predictor(yolo_weight_path, device=device)
        self.unet_predictor = UNet_Predictor(unet_weight_path, bilinear=bilinear, device=device)
        self.crop_size = crop_size
    def predict(self, img_path: str):
        im = cv2.imread(img_path)
        im_h, im_w , c = im.shape
        crop_imgs, box_infos, cls_idx = self.yolov5_predictor.predict_crop(img_path)
        cls_name = class_names[cls_idx]
        predicted_mask = np.zeros((im_h, im_w))
        for crop_img, box_info in zip(crop_imgs, box_infos):
            box_w, box_h, x1, y1, x2, y2 = box_info
            origin_size = (box_w, box_h)

            resized_crop_img = cv2.resize(crop_img, self.crop_size) 
            predict_mask = self.unet_predictor.predict(cv2.cvtColor(resized_crop_img, cv2.COLOR_BGR2RGB), to_img=True) # Unet use Pillow, so we need to convert from BGR to RGB
            predict_mask = cv2.resize(predict_mask, origin_size, interpolation=cv2.INTER_NEAREST)
            predicted_mask[y1:y2, x1:x2] = predict_mask
        return predicted_mask
    

c:\Users\user\anaconda3\envs\yolov5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import cv2
import numpy as np

yolo_weight = './yolov5/runs/train/[2022-12-29-1549]960_bs16_med_E400/weights/best.pt'
unet_weight = './UNet/ckp_fold2/checkpoint_epoch40.pth'
device = 'cuda'
predictor = Powder_Spreading_Predictor(yolo_weight, unet_weight, device=device, bilinear=True)

Fusing layers... 
Model summary: 212 layers, 20861016 parameters, 0 gradients, 47.9 GFLOPs


In [8]:
import os
from os.path import join
from tqdm import tqdm
split = 'Val'
data_root = './data/yolov5/{}'.format(split)

dice_scores = []
for img_name in tqdm(os.listdir(join(data_root, 'images'))):
    img_path = join(data_root, 'images', img_name)
    mask_path = join(data_root, 'masks', img_name)
    
    predicted_mask = predictor.predict(img_path)
    gt_mask = load_img(mask_path ,binary=True)
    dice_scores.append(dice_coeff(predicted_mask, gt_mask))
print(np.mean(dice_scores))

100%|██████████| 150/150 [01:02<00:00,  2.40it/s]

0.9096528643850424


In [ ]:
import os
from os.path import join
from tqdm import tqdm
split = 'Val'
data_root = './data/yolov5/{}'.format(split)

dice_scores = []
for img_name in tqdm(os.listdir(join(data_root, 'images'))):
    img_path = join(data_root, 'images', img_name)
    mask_path = join(data_root, 'masks', img_name)
    
    predicted_mask = predictor.predict(img_path)
    gt_mask = load_img(mask_path ,binary=True)
    dice_scores.append(dice_coeff(predicted_mask, gt_mask))
print(np.mean(dice_scores))

In [ ]:
import matplotlib.pyplot as plt
import os
from os.path import join
%matplotlib inline



predictor.unet_predictor.threshold = 0.5
split = 'Val'


for i in range(10, 20):
    img_name = '/scratch_{}.png'.format(i)

    mask_path = './data/yolov5/{}/masks/{}'.format(split, img_name)
    img_path = './data/yolov5/{}/images/{}'.format(split, img_name)

    label_txt = './data/unet/{}/labels/{}.txt'.format(split, img_name.split('.')[0])
    with open(label_txt, 'r') as f:
        lines = f.readlines()
    
    predicted_mask = predictor.predict(img_path)
    gt_mask = load_img(mask_path, binary=True)


    print(dice_coeff(predicted_mask, gt_mask))
    #   
    # im_h, im_w = result.shape
    # gt_unet_mask = np.zeros((im_h, im_w))
    # for line in lines:
    #     mask_name = line.split(',')[0]
    #     box_w, box_h, x1, y1, x2, y2 = [int(c) for c in line.split(',')[1:]]
    #     mask = load_img('./data/unet/{}/masks/{}'.format(split, mask_name), True)
    #     mask = cv2.resize(mask, (box_w, box_h), interpolation=cv2.INTER_NEAREST)
    #     gt_unet_mask[y1:y2, x1:x2] = mask
    

    # gt_mask = load_img(mask_path, binary=True)

    # ax = plt.subplot(1,3,1)
    # ax.imshow(result)

    # ax = plt.subplot(1,3,2)
    # ax.imshow(gt_unet_mask)


    # ax = plt.subplot(1,3,3)
    # ax.imshow(gt_mask)

    # plt.show()

In [ ]:
yolov5_predictor = Yolov5_Predictor(yolo_weight, device=device)
unet_predictor = UNet_Predictor(unet_weight, device=device)

In [ ]:
im = cv2.imread(img_path)
im_h, im_w , c = im.shape
crop_imgs, box_infos = yolov5_predictor.predict_crop(img_path)
predicted_mask = np.zeros((im_h, im_w))

In [ ]:
resized_size = (320, 320)
for crop_img, box_info in zip(crop_imgs, box_infos):
    box_w, box_h, x1, y1, x2, y2 = box_info
    origin_size = (box_w, box_h)
    predict_mask = unet_predictor.predict(cv2.cvtColor(cv2.resize(crop_img, resized_size), cv2.COLOR_BGR2RGB), to_img=True)

    plt.imshow(predict_mask)
    plt.show()

In [3]:
%cd UNet

d:\研究所課程\孫永年影像處理\homework\ncku_image_processing\final_hw\UNet\UNet


In [6]:
!python my_train_k_fold.py --k-fold 3 --epoch 60 --bilinear

python: can't open file 'my_train_k_fold.py': [Errno 2] No such file or directory


In [ ]:
! python my_train.py --k-fold 4 --epoch 60 --bilinear